# vCAT MVPA - LSS Task 
## Create condition-specific EV files
### Task structure: 2 sets/2 runs per set
#### 180 trials/run: *30 BL, *50 cond, *100 fixed

In [37]:
import os
from os.path import join
import numpy as np
import pandas as pd
from glob import glob

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
subs1 = ['sub-005']
stim_sets = ['set1', 'set2']

for sub in subs:
    sub_dir = '/home/data/madlab/Mattfeld_vCAT/behav/{0}/'.format(sub)
    dir_files = sorted(glob(join(sub_dir, '{0}_simp_task*.csv'.format(sub)))) 
    for i, curr_set in enumerate(stim_sets):
        for curr_run in ['1', '2']:
            if curr_run == '1':
                run = pd.read_csv(dir_files[i*2]) #create dataframe for text files to extract EVs
            else:
                run = pd.read_csv(dir_files[i*2+1])              

            trialtype = run['trialtype'].values #convert dataframes into numpy arrays
            acc = run['acc'].values
            resp = run['resp'].values
            stim = run['stim'].values
            onsets = run['onset'].values
             
            trial_shift = trialtype[1:] #shift trial type back and insert dummy (-1) in last index
            trial_shift = np.append(trial_shift, -1)
            acc_shift = acc[1:] #shift accuracy back and insert dummy (-1) in last index
            acc_shift = np.append(acc_shift, -1)
            resp_shift = resp[1:] #shift response back and insert dummy (-1) in last index
            resp_shift = np.append(resp_shift, -1)
                       
            #grab indices matching specified criteria
            fixed_BL_face = np.where((trialtype=='fixed')&(trial_shift=='BL')&((stim=='face1')|(stim=='face2')))[0]           
            fixed_BL_scene = np.where((trialtype=='fixed')&(trial_shift=='BL')&((stim=='scene1')|(stim=='scene2')))[0]
            remaining = list(range(0,180)) #remaining contains trials which dont proceed corr/incorr conds, and nonresp  

            for x in fixed_BL_face:
                remaining.remove(x) #remove fixed before face trials from all remaining
                remaining.remove(x+1) #remove following BL trials from all remaining
            for x in fixed_BL_scene:
                remaining.remove(x) #remove fixed before scene trials from all remaining
                remaining.remove(x+1) #remove following BL trials from all remaining
                
            #index onsets array using indices from np.where() criteria 
            fixed_BL_face_onsets = onsets[fixed_BL_face] 
            fixed_BL_scene_onsets = onsets[fixed_BL_scene] 
            remaining_onsets = onsets[remaining]
            onsets2 = [] #onsets with combined fixed-BL pairs for mtrx
            onset_durations = [] #need to arrange different durations by trial order
            for y, curr_trial in enumerate(trialtype): #iterate through trialtypes 
                if curr_trial == 'fixed' and trial_shift[y] == 'BL': #if fixed followed by BL, assign duration of 8.5 sec
                    onsets2.append(onsets[y]) #append fixed/BL onset
                    onset_durations.append(8.5) #append duration of fixed and BL trial pairs
                elif curr_trial == 'BL': #pass if current trialtype is BL
                    continue
                else: # if not fixed followed by BL or BL, assign duration of 3.0 sec
                    onsets2.append(onsets[y])
                    onset_durations.append(3.0) #append duration of fixed not followed by BL and cond trials
         
            #vstack matrix containing *ALL* onsets, durations, and amplitudes in vertical columns 
            mtrx = np.vstack((onsets2, onset_durations, np.ones(len(onsets2)))).T 
            fixed_BL_face_mtrx = np.vstack((fixed_BL_face_onsets,  np.ones(len(fixed_BL_face_onsets))*8.5,
                                            np.ones(len(fixed_BL_face_onsets)))).T 
            fixed_BL_scene_mtrx = np.vstack((fixed_BL_scene_onsets,  np.ones(len(fixed_BL_scene_onsets))*8.5,
                                             np.ones(len(fixed_BL_scene_onsets)))).T             
            remaining_mtrx = np.vstack((remaining_onsets, np.ones(len(remaining_onsets))*3.0,
                                        np.ones(len(remaining_onsets)))).T

            if not os.path.exists(join(sub_dir, 'MVPA-LSS')): #if directory does not exist
                os.makedirs(join(sub_dir, 'MVPA-LSS')) #create it
                
            if curr_run == '1': #if first run in stim set
                np.savetxt(join(sub_dir, 'MVPA-LSS', 'run{0}.txt'.format(i*2+1)), mtrx, delimiter='\t', fmt='%.4f')                
                for trial in ['fixed_BL_face', 'fixed_BL_scene', 'remaining']: #for all trial types
                    exec('np.savetxt(sub_dir+"MVPA-LSS/"+"run{0}_{1}.txt",{1}_mtrx,delimiter="\t",fmt="%.4f")'.format(i*2+1,trial))
            else: #if second run in stim set
                np.savetxt(join(sub_dir, 'MVPA-LSS', 'run{0}.txt'.format(i*2+2)), mtrx, delimiter='\t', fmt='%.4f')                
                for trial in ['fixed_BL_face', 'fixed_BL_scene', 'remaining']:
                    exec('np.savetxt(sub_dir+"MVPA-LSS/"+"run{0}_{1}.txt",{1}_mtrx,delimiter="\t",fmt="%.4f")'.format(i*2+2,trial))


sub-005

sub-006

sub-007

sub-008

sub-010

sub-012

sub-013

sub-014

sub-015

sub-016

sub-018

sub-019

sub-020

sub-021

sub-022

sub-023

sub-024

sub-025

sub-026

sub-027

sub-028

sub-029

sub-030

sub-031

sub-032
